# 前準備

gensimのバージョンアップといったん再起動が必要

In [ ]:
!pip install --upgrade gensim

import os
os.kill(os.getpid(), 9)

mecabのインストール

In [ ]:
!apt-get install mecab mecab-ipadic-utf8
!pip install mecab-python3
!ln -s  /etc/mecabrc /usr/local/etc/mecabrc

# 文書セットの準備

In [ ]:
sentences = ['吾輩は猫である名前は',
             '猫はあるはある',
             '吾輩は猫では名前では',
             '猫はであるはあるはあるあるある',
             '名前ではある名前ではある名前ではある',
             'はであるはある']

In [ ]:
import MeCab

tagger = MeCab.Tagger("-Owakati")

# 各sentence（ドキュメント）に対して分かち書き
tokenized_docs = [tagger.parse(sentence).split() for sentence in sentences]
for d in tokenized_docs:
  print(d)

# 文書のベクトル表現（BoW）

## 単語文書行列の作成

In [ ]:
import gensim
import seaborn

# 単語のリスト（辞書）をつくる。IDと語のmapping
dic = gensim.corpora.Dictionary(tokenized_docs)
print(dic)

# 分かち書きされた各ドキュメントをBoWベクトル（頻度）にする
BOW_vecs = [dic.doc2bow(tokenized_doc) for tokenized_doc in tokenized_docs]
print(BOW_vecs)




## 文書間の類似度の可視化

In [ ]:
# 各ドキュメントとの類似度を求めるための準備
index = gensim.similarities.MatrixSimilarity(BOW_vecs, num_features=len(dic))
# 各ドキュメント(BOW_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
similarities = [index[BOW_vec] for BOW_vec in BOW_vecs]
seaborn.heatmap(similarities, annot=True, cmap='Blues')

# LSI による行列分解

In [ ]:
# lsiを行い、ドキュメントベクトル（topic分布ベクトル）を求める
lsimodel = gensim.models.LsiModel(BOW_vecs, id2word=dic, num_topics=2)
lsi_vecs = [lsimodel[BOW_vec] for BOW_vec in BOW_vecs]
print(lsi_vecs)



## 文書間の類似度の可視化


In [ ]:
# 各ドキュメントとの類似度を求めるための準備
index = gensim.similarities.MatrixSimilarity(lsi_vecs, num_features=len(dic))
# 各ドキュメント(lsi_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
similarities = [index[lsi_vec] for lsi_vec in lsi_vecs]
seaborn.heatmap(similarities, annot=True, cmap='Blues')


## 分解後の各行列の確認

In [ ]:
# 左特異行列U
u = lsimodel.projection.u
print(u)

# 特異値（対角成分だけのベクトル）
sigma = lsimodel.projection.s
print(sigma)


lsimodel.show_topics()